In [ ]:
!python --version

In [ ]:
!git clone https://github.com/AarohiSingla/Tracking-and-counting-Using-YOLOv8-and-DeepSORT.git


In [ ]:
!cd Tracking-and-counting-Using-YOLOv8-and-DeepSORT
!pip install ultralytics

In [2]:
import ultralytics
ultralytics.__version__

c:\Users\MANAK\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'8.0.221'

In [3]:
import torch
torch.__version__

'1.10.0+cpu'

In [ ]:
torch.cuda.get_device_name(0)

# Detect, track and count Persons

In [4]:
%cd Tracking-and-counting-Using-YOLOv8-and-DeepSORT

[WinError 2] The system cannot find the file specified: 'Tracking-and-counting-Using-YOLOv8-and-DeepSORT'
c:\Users\MANAK\Desktop\CES751PROJ\Tracking-and-counting-Using-YOLOv8-and-DeepSORT


c:\Users\MANAK\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [1]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

results = model("car.jpg", save=True)



class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    probs = result.probs  # Class probabilities for classification outputs
    cls = boxes.cls.tolist()  # Convert tensor to list
    xyxy = boxes.xyxy
    xywh = boxes.xywh  # box with xywh format, (N, 4)
    conf = boxes.conf
    print(cls)
    for class_index in cls:
        class_name = class_names[int(class_index)]
        print("Class:", class_name)

c:\Users\MANAK\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



image 1/1 c:\Users\MANAK\Desktop\CES751PROJ\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\car.jpg: 288x640 1 car, 140.0ms
Speed: 13.5ms preprocess, 140.0ms inference, 15.7ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs\detect\predict12
[2.0]
Class: car


# DeepSORT

In [ ]:
!ls

In [27]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker


deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=10)

In [28]:
# Define the video path
video_path = 'india.MOV'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output3.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(cap.read())

In [29]:
frames = []

unique_track_ids = set()

In [31]:
i = 0  #moving average prediction of motion implemented linear regression commented out cus not useful
pcount=0 #no of predicted conflict points in a frame
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()
g_time=time.perf_counter()
box_arrays={} #dictionary for bounding box of each id storing bounding boxes for all frames
my_counter=0 #no of frames
n=2 #prediction how many frames ahead, has to be small in comparison the coordinates considered for linear reg. Here we have considered only 5 coordinates so do a small no
prediction_boxes={}
prediction_points={}
def estimate_coef(x, y):
    # number of observations/points
    n = np.size(x)

    # mean of x and y vector
    m_x = np.mean(x)
    m_y = np.mean(y)

    # calculating cross-deviation and deviation about x
    SS_xy = np.sum(y*x) - n*m_y*m_x
    SS_xx = np.sum(x*x) - n*m_x*m_x

    # calculating regression coefficients
    b_1 = SS_xy / SS_xx
    b_0 = m_y - b_1*m_x

    return (b_0, b_1)

while cap.isOpened():


    ret, frame = cap.read()
    if not ret:
      cap.release()
    if ret:

        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = og_frame.copy()

        model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)


        results = model(frame, device, classes=(0,1,2,3,5,7), conf=0.5)

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        print(results)
        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)

        tracks = tracker.update(bboxes_xywh, conf, og_frame)

        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height

            if track_id in box_arrays:
              box_arrays[track_id].append([x1,y1,w,h])
            else:
              box_arrays[track_id]=[]
              box_arrays[track_id].append([x1,y1,w,h])




            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1+w/2), int(y1+h/2)), (int(x1 + w/2+2), int(y1 + h/2+2)), color, 2) 
            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2) 
            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)

        for id in box_arrays:
              cid = id % 3
              if cid == 0:
                color = red_color
              elif cid == 1:
                color = blue_color
              else:
                color = green_color

              if len(box_arrays[id])>=5:
               x12=np.array([box_arrays[id][len(box_arrays[id])-5][0],box_arrays[id][len(box_arrays[id])-4][0],box_arrays[id][len(box_arrays[id])-3][0],box_arrays[id][len(box_arrays[id])-2][0],box_arrays[id][len(box_arrays[id])-1][0]])
               y12=np.array([box_arrays[id][len(box_arrays[id])-5][1],box_arrays[id][len(box_arrays[id])-4][1],box_arrays[id][len(box_arrays[id])-3][1],box_arrays[id][len(box_arrays[id])-2][1],box_arrays[id][len(box_arrays[id])-1][1]])
               w2=np.array([box_arrays[id][len(box_arrays[id])-5][2],box_arrays[id][len(box_arrays[id])-4][2],box_arrays[id][len(box_arrays[id])-3][2],box_arrays[id][len(box_arrays[id])-2][2],box_arrays[id][len(box_arrays[id])-1][2]])
               h2=np.array([box_arrays[id][len(box_arrays[id])-5][3],box_arrays[id][len(box_arrays[id])-4][3],box_arrays[id][len(box_arrays[id])-3][3],box_arrays[id][len(box_arrays[id])-2][3],box_arrays[id][len(box_arrays[id])-1][3]])
               c=np.array([1,2,3,4,5])

               ax12,bx12=estimate_coef(c,x12) #Y=a+bX
               ay12,by12=estimate_coef(c,y12)
               aw2,bw2=estimate_coef(c,w2)
               ah2,bh2=estimate_coef(c,h2)

               if id in prediction_boxes:
                  prediction_boxes[id]=[ax12+bx12*(5+n),ay12+by12*(5+n),aw2+bw2*(5+n),ah2+bh2*(5+n)]
               else:
                  prediction_boxes[id]=[]
                  prediction_boxes[id]=[ax12+bx12*(5+n),ay12+by12*(5+n),aw2+bw2*(5+n),ah2+bh2*(5+n)]

               #cv2.rectangle(og_frame, (int(prediction_boxes[id][0]), int(prediction_boxes[id][1])), (int(prediction_boxes[id][0] + prediction_boxes[id][2]), int(prediction_boxes[id][1] + prediction_boxes[id][3])), color, 2)
        for id in prediction_boxes:
           if id in prediction_points:
            
            prediction_points[id]=[int(prediction_boxes[id][0] + prediction_boxes[id][2]/2), int(prediction_boxes[id][1] + prediction_boxes[id][3]/2)]
           else:
              prediction_points[id]=[]
              prediction_points[id]=[int(prediction_boxes[id][0] + prediction_boxes[id][2]/2), int(prediction_boxes[id][1] + prediction_boxes[id][3]/2)]
        
        for ip in range(0,len(prediction_points)):
           for kip in range(ip, len(prediction_points)):
              xx1=prediction_points[list(prediction_points.keys())[ip]][0]
              xx2=prediction_points[list(prediction_points.keys())[kip]][0]
              yy1=prediction_points[list(prediction_points.keys())[ip]][1]
              yy2=prediction_points[list(prediction_points.keys())[kip]][1]
              
              if int(((xx1-xx2)^2+(yy1-yy2)^2))<1: # 1 is the distance between two predicted points for them to be in conflict
                 cv2.rectangle(og_frame, (int((xx1+xx2)/2), int((yy1+yy2)/2)), (int((xx1+xx2)/2), int((yy1+yy2)/2)), color, 2)
                 pcount+=1
                 
                 

        print(pcount)
        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)
        pcount=0
        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        my_counter+=1
        print(my_counter)


        # Draw person count on frame
        cv2.putText(og_frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))


        if (current_time-g_time)>60:
          cap.release()
        # Show the frame
        #cv2.imshow("Video", og_frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break


for i in tracker.tracker.tracks:
  print(i.track_id)
cap.release()
out.release()
#cv2.destroyAllWindows()


<generator object BasePredictor.stream_inference at 0x000001F105162E40>

0: 384x640 2 persons, 8 cars, 1 motorcycle, 140.1ms
Speed: 0.0ms preprocess, 140.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0
1
<generator object BasePredictor.stream_inference at 0x000001F105175040>

0: 384x640 2 persons, 8 cars, 1 motorcycle, 137.0ms
Speed: 0.0ms preprocess, 137.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0
2
<generator object BasePredictor.stream_inference at 0x000001F105175190>

0: 384x640 2 persons, 7 cars, 1 motorcycle, 124.9ms
Speed: 0.0ms preprocess, 124.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0
3
<generator object BasePredictor.stream_inference at 0x000001F1270C0040>

0: 384x640 2 persons, 8 cars, 1 motorcycle, 124.2ms
Speed: 0.0ms preprocess, 124.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
0
4
<generator object BasePredictor.stream_inference at 0x000001F11B05A350>

0: 384x640 2 person